# Notebook3 - Silver -> Gold/Curated

## 1 - Configurações iniciais:

In [0]:
# Configuração do ambiente
from pyspark.sql import functions as F
spark.sql("CREATE SCHEMA IF NOT EXISTS medalhao.gold")
spark.sql("USE CATALOG medalhao")
spark.sql("USE SCHEMA gold")

print("Configuração do ambiente Gold concluída")

- Criação do schema "gold" caso o mesmo não exista
- Definição do catálogo "medalhao" como padrão
- Deixa selecionado o schema "gold" para uso no notebook

## 2 - Projeto 1 - Área de Logística(Vendas por Localidade):

### 2.1 - Criação da Tabela "gold.ft_vendas_consumidor_local":

In [0]:
# Leitura das tabelas silver necessárias
ft_pedido_total_silver = spark.table("medalhao.silver.ft_pedido_total")
ft_consumidores_silver = spark.table("medalhao.silver.ft_consumidores")

# Criação da tabela fato
gold_ft_vendas_consumidor_local = (
    ft_pedido_total_silver
    .join(ft_consumidores_silver, "id_consumidor", "inner")
    .select(
        F.col("id_pedido"),
        F.col("id_consumidor"),
        F.col("valor_total_pago_brl").alias("valor_total_pedido_brl"),
        F.col("cidade"),
        F.col("estado"),
        F.col("data_pedido")
    )
)

# Salvamento da tabela
spark.sql("DROP TABLE IF EXISTS medalhao.gold.ft_vendas_consumidor_local")
gold_ft_vendas_consumidor_local.write.format("delta").mode("overwrite").saveAsTable("medalhao.gold.ft_vendas_consumidor_local")

- Junta as informações contidas na tabela "ft_pedido_total_silver" com 2 colunas de "ft_consumidores_silver"
- Seleciona apenas as colunas do .select e organiza na ordem presente no pdf da atividade3
- Salva a tabela "ft_vendas_consumidor_local"

### 2.2 - Criação da view "gold.view_total_compras_por_consumidor":

In [0]:
# Criação da view consolidada por localidade
spark.sql("""
CREATE OR REPLACE VIEW medalhao.gold.view_total_compras_por_consumidor AS
SELECT 
    cidade,
    estado,
    COUNT(DISTINCT id_pedido) as quantidade_vendas,
    SUM(valor_total_pedido_brl) as valor_total_localidade
FROM medalhao.gold.ft_vendas_consumidor_local
GROUP BY cidade, estado
ORDER BY valor_total_localidade DESC
""")

# Validação da view
view_result = spark.sql("SELECT * FROM medalhao.gold.view_total_compras_por_consumidor LIMIT 10")
display(view_result)

# Consulta total de vendas por estado
vendas_por_estado = spark.sql("""
SELECT 
    estado,
    SUM(quantidade_vendas) as total_vendas,
    SUM(valor_total_localidade) as valor_total_estado
FROM medalhao.gold.view_total_compras_por_consumidor
GROUP BY estado
ORDER BY valor_total_estado DESC
""")

display(vendas_por_estado)

- Criação, ou reposição, da view com nome indicado, selecionando as colunas de cidade, estado, contando quantidade_vendas e valor_total_localidade(somando os valores de valor_total_pedido_brl), vindos ta tabela criada anteriormente 
- Realização do agrupamento das vendas por cidade e respectivo estado
- Ordena levando em conta os preços da coluna valor_total_localidade, do maior até o menor
- Agrupamento dos dados da view pelo estado
- Ordena pela coluna valor_total_estado(vinda da soma devalor_total_localidade), do maior até o menor

## 3 - Projeto 2 - Área de Logística (Análise de Atrasos de Entregas):


### 3.1 - Criação da tabela "gold.ft_atrasos_pedidos_local_vendedor":

In [0]:
# Leitura das tabelas silver necessárias
ft_pedidos_silver = spark.table("medalhao.silver.ft_pedidos")
ft_consumidores_silver = spark.table("medalhao.silver.ft_consumidores")
ft_itens_pedidos_silver = spark.table("medalhao.silver.ft_itens_pedidos")

# Criação da tabela de atrasos
gold_ft_atrasos_pedidos_local_vendedor = (
    ft_pedidos_silver
    .join(ft_consumidores_silver, "id_consumidor")
    .join(ft_itens_pedidos_silver.select("id_pedido", "id_vendedor"), "id_pedido")
    .select(
        F.col("id_pedido"),
        F.col("id_vendedor"),
        F.col("id_consumidor"),
        F.col("entrega_no_prazo"),
        F.col("tempo_entrega_dias"),
        F.col("tempo_entrega_estimado_dias"),
        F.col("cidade"),
        F.col("estado")
    )
)

# Salvamento da tabela
spark.sql("DROP TABLE IF EXISTS medalhao.gold.ft_atrasos_pedidos_local_vendedor")
gold_ft_atrasos_pedidos_local_vendedor.write.format("delta").mode("overwrite").saveAsTable("gold.ft_atrasos_pedidos_local_vendedor")

- Junta as informações contidas na tabela "ft_pedidos_silver" com 1 coluna de "ft_consumidores_silver" e 2 de "ft_itens_pedidos_silver"
- Seleciona apenas as colunas do .select e organiza na ordem presente no pdf da atividade3
- Salva a tabela "ft_atrasos_pedidos_local_vendedor"

### 3.2 - Criação das Views Analíticas:

In [0]:
#3.2.1 - "medalhao.gold.view_tempo_medio_entrega_localidade":
# Criação da view para tempo médio de entrega por localidade
spark.sql("""
CREATE OR REPLACE VIEW medalhao.gold.view_tempo_medio_entrega_localidade AS
SELECT 
    cidade,
    estado,
    CAST(ROUND(AVG(tempo_entrega_dias), 2) AS INT) as tempo_medio_entrega,
    CAST(ROUND(AVG(tempo_entrega_estimado_dias), 2) AS INT) as tempo_medio_estimado,
    CASE 
        WHEN AVG(tempo_entrega_dias) > AVG(tempo_entrega_estimado_dias) THEN 'SIM'
        ELSE 'NÃO'
    END as entrega_maior_que_estimado
FROM medalhao.gold.ft_atrasos_pedidos_local_vendedor
GROUP BY cidade, estado
""")

# Validação da view
tempo_medio_view = spark.sql("SELECT * FROM medalhao.gold.view_tempo_medio_entrega_localidade LIMIT 10")
display(tempo_medio_view)

# 3.2.2 - "gold.view_vendedor_pontualidade":
# Criação da view para análise de pontualidade por vendedor
spark.sql("""
CREATE OR REPLACE VIEW medalhao.gold.view_vendedor_pontualidade AS
WITH pedidos_vendedor AS (
    SELECT 
        id_vendedor,
        id_pedido,
            MAX(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) as pedido_atrasado
        FROM medalhao.gold.ft_atrasos_pedidos_local_vendedor
        GROUP BY id_vendedor, id_pedido
),
estatisticas_vendedor AS (
    SELECT 
        id_vendedor,
        COUNT(DISTINCT id_pedido) as total_pedidos,
        SUM(pedido_atrasado) as total_atrasados
    FROM pedidos_vendedor
    GROUP BY id_vendedor
)
SELECT 
    id_vendedor,
    total_pedidos,
    total_atrasados,
    ROUND(
        (total_atrasados * 100.0 / total_pedidos), 2
    ) as percentual_atraso
FROM estatisticas_vendedor
ORDER BY percentual_atraso DESC
""")

# Validação da view
pontualidade_view = spark.sql("SELECT * FROM medalhao.gold.view_vendedor_pontualidade LIMIT 10")
display(pontualidade_view)

3.2.1 - "medalhao.gold.view_tempo_medio_entrega_localidade":
- Realização do agrupamento das vendas por cidade e respectivo estado
- Criação, ou reposição, da view com nome indicado, selecionando as colunas de cidade, estado, contando tempo_medio_entrega e tempo_medio_estimado, de modo a manter o número inteiro de dias
- Usa as tabelas de tempo para identificar os dados da coluna entrega_maior_que_estimado, com "Sim" ou Não".

3.2.2 - "gold.view_vendedor_pontualidade":
- No primeiro bloco ocorre o agrupamento dos dados da view pelo vendedor e pedido(usa dados da view anterior)
- Usa o MAX pois se apenas um pedido do todo estiver atrasado, o pedido todo está também
- No segundo bloco o agrupamento é feito pelo is_vendedor, contando quantos pedidos cada vendedor possui(DISTINCT para evitar duplicação) e soma os pedidos atrasado do venderor
- No último bloco a ordenação ocorre pela percentual_atraso e o calculo desse percentual é feito a partir da divisaõ do número de pedidos atrasados pelo número de pedidos do vendedor, multiplica por 100 e garante 2 casas decimais

## 4 - Projeto 3 - Área Comercial (Análises de Vendas por Período):


### 4.1 - Criação da Dimensão de Tempo "gold.dm_tempo":

In [0]:
# Definição do período (baseado nos dados disponíveis)
data_inicio = "2016-01-01"
data_fim = "2024-12-31"

# Criação da dimensão tempo usando sequence e explode
dm_tempo = (
    spark.sql(f"""
    SELECT explode(sequence(to_date('{data_inicio}'), to_date('{data_fim}'), interval 1 day)) as sk_tempo
    """)
    .select(
        F.col("sk_tempo"),
        F.year("sk_tempo").alias("ano"),
        F.quarter("sk_tempo").alias("trimestre"),
        F.month("sk_tempo").alias("mes"),
        F.weekofyear("sk_tempo").alias("semana_do_ano"),
        F.dayofmonth("sk_tempo").alias("dia"),
        F.dayofweek("sk_tempo").alias("dia_da_semana_num"),
        F.date_format("sk_tempo", "EEEE").alias("dia_da_semana_nome"),
        F.date_format("sk_tempo", "MMMM").alias("mes_nome"),
        F.when(F.dayofweek("sk_tempo").isin(1, 7), "Sim").otherwise("Não").alias("fim_de_semana")
    )
)

dm_tempo = dm_tempo.withColumn(
    "dia_da_semana_nome",
    F.when(F.col("dia_da_semana_num") == 1, "Domingo")
     .when(F.col("dia_da_semana_num") == 2, "Segunda")
     .when(F.col("dia_da_semana_num") == 3, "Terça")
     .when(F.col("dia_da_semana_num") == 4, "Quarta")
     .when(F.col("dia_da_semana_num") == 5, "Quinta")
     .when(F.col("dia_da_semana_num") == 6, "Sexta")
     .when(F.col("dia_da_semana_num") == 7, "Sábado")
)

dm_tempo = dm_tempo.withColumn(
    "mes_nome",
    F.when(F.col("mes") == 1, "Janeiro")
     .when(F.col("mes") == 2, "Fevereiro")
     .when(F.col("mes") == 3, "Março")
     .when(F.col("mes") == 4, "Abril")
     .when(F.col("mes") == 5, "Maio")
     .when(F.col("mes") == 6, "Junho")
     .when(F.col("mes") == 7, "Julho")
     .when(F.col("mes") == 8, "Agosto")
     .when(F.col("mes") == 9, "Setembro")
     .when(F.col("mes") == 10, "Outubro")
     .when(F.col("mes") == 11, "Novembro")
     .when(F.col("mes") == 12, "Dezembro")
)

# Salvamento da tabela
spark.sql("DROP TABLE IF EXISTS medalhao.gold.dm_tempo")
dm_tempo.write.format("delta").mode("overwrite").saveAsTable("gold.dm_tempo")

- Define-se um intervalo de datas a serem analisadas baseado nos dados disponíveis
- O sequence cria uma lista de datas em sequencia(com intervalo de um dia), enquanto o explode transforma cada data da lista em uma linha da coluna sk_tempo
- Conversão do título de cada coulna para o título presente no pdf, verificação dos dias que são finais de semana
- Identificação de cada número que representa o dia da semana

### 4.2 - Criação da Fato "gold.ft_vendas_geral":

In [0]:
ft_itens_pedidos_silver = spark.table("medalhao.silver.ft_itens_pedidos")
ft_pedidos_silver = spark.table("medalhao.silver.ft_pedidos")
ft_produtos_silver = spark.table("medalhao.silver.ft_produtos")
ft_avaliacoes_silver = spark.table("medalhao.silver.ft_avaliacoes_pedidos")
dm_cotacao_silver = spark.table("medalhao.silver.dm_cotacao_dolar")

# Preparação da tabela de avaliações
avaliacoes_agg = (
    ft_avaliacoes_silver
    .groupBy("id_pedido")
    .agg(F.avg("avaliacao").alias("avaliacao_pedido"))
)

# Criação da fato vendas geral
gold_ft_vendas_geral = (
    ft_itens_pedidos_silver
    .join(ft_pedidos_silver.select("id_pedido", "id_consumidor", "status", "tempo_entrega_dias", "entrega_no_prazo", "pedido_compra_timestamp"), "id_pedido")
    .join(ft_produtos_silver.select("id_produto", "categoria_produto"), "id_produto", "left")
    .join(avaliacoes_agg, "id_pedido", "left")
    .join(
        dm_cotacao_silver.select("data", "cotacao_dolar"),
        F.to_date("pedido_compra_timestamp") == F.col("data"),
        "left"
    )
    .select(
        F.col("id_pedido"),
        F.col("id_item"),
        F.col("id_consumidor").alias("fk_cliente"),
        F.col("id_produto").alias("fk_produto"),
        F.col("id_vendedor").alias("fk_vendedor"),
        F.to_date("pedido_compra_timestamp").alias("fk_tempo"),
        F.col("status").alias("status_pedido"),
        F.col("tempo_entrega_dias"),
        F.col("entrega_no_prazo"),
        F.col("preco_BRL").alias("valor_produto_brl"),
        F.col("preco_frete").alias("valor_frete_brl"),
        (F.col("preco_BRL") + F.col("preco_frete")).alias("valor_total_item_brl"),
        (F.col("preco_BRL") / F.coalesce(F.col("cotacao_dolar"), F.lit(1))).cast("decimal(12,2)").alias("valor_produto_usd"),
        (F.col("preco_frete") / F.coalesce(F.col("cotacao_dolar"), F.lit(1))).cast("decimal(12,2)").alias("valor_frete_usd"),
        ((F.col("preco_BRL") + F.col("preco_frete")) / F.coalesce(F.col("cotacao_dolar"), F.lit(1))).cast("decimal(12,2)").alias("valor_total_item_usd"),
        F.coalesce(F.col("cotacao_dolar"), F.lit(1)).alias("cotacao_dolar"),
        F.coalesce(F.col("avaliacao_pedido"), F.lit(0)).alias("avaliacao_pedido").cast("decimal(3,2)")
    )
)

# Salvamento
spark.sql("DROP TABLE IF EXISTS medalhao.gold.ft_vendas_geral")
gold_ft_vendas_geral.write.format("delta").mode("overwrite").saveAsTable("medalhao.gold.ft_vendas_geral")

display(gold_ft_vendas_geral.limit(10))

- Combina 5 tabelas da camada silver para junta-las 
- Agrega as avaliações de acordo com o id do pedido e tira sua média
- Usa ft_itens_pedidos_silver de base para juntar outras colunas das tabelas trazidas
- Organiza na ordem presente no pdf, e nas colunas de valor_total_item_brl até valor_total_item_usd ocorrem também calculos para achar os valores pedidos, também são passados para o tipo de dado pedido na atividade

### 4.3 - Criação da view "gold.view_vendas_por_periodo":

In [0]:
# View analítica temporal
spark.sql("""
CREATE OR REPLACE VIEW medalhao.gold.view_vendas_por_periodo AS
SELECT 
    dt.ano,
    dt.trimestre,
    dt.mes,
    dt.mes_nome,
    dt.dia,
    dt.dia_da_semana_num,
    COUNT(DISTINCT fv.id_pedido) as total_pedidos,
    COUNT(fv.id_item) as total_itens,
    CAST(ROUND(SUM(fv.valor_total_item_brl), 2) AS DECIMAL(12,2)) as receita_total_brl,
    CAST(ROUND(SUM(fv.valor_total_item_usd), 2) AS DECIMAL(12,2)) as receita_total_usd,
    CAST(ROUND(AVG(fv.valor_total_item_brl), 2) AS DECIMAL(12,2)) as ticket_medio_brl,
    CAST(ROUND(AVG(fv.avaliacao_pedido), 2) AS DECIMAL(3,2)) as avaliacao_media
FROM medalhao.gold.ft_vendas_geral fv
JOIN medalhao.gold.dm_tempo dt ON fv.fk_tempo = dt.sk_tempo
WHERE fv.status_pedido = 'entregue'
GROUP BY 
    dt.ano, dt.trimestre, dt.mes, dt.mes_nome, dt.dia, dt.dia_da_semana_num
""")

# Validação da view
vendas_periodo_view = spark.sql("SELECT * FROM medalhao.gold.view_vendas_por_periodo LIMIT 10")
display(vendas_periodo_view)

# 4.3.1 - Consultas Analíticas
# Dia da semana com maior receita
dia_maior_receita = spark.sql("""
SELECT 
    CASE dia_da_semana_num
        WHEN 1 THEN 'Domingo'
        WHEN 2 THEN 'Segunda'
        WHEN 3 THEN 'Terça'
        WHEN 4 THEN 'Quarta'
        WHEN 5 THEN 'Quinta'
        WHEN 6 THEN 'Sexta'
        WHEN 7 THEN 'Sábado'
    END AS dia_da_semana_nome,
    SUM(receita_total_brl) as receita_total
FROM medalhao.gold.view_vendas_por_periodo
GROUP BY dia_da_semana_num
ORDER BY receita_total DESC
LIMIT 1
""")

print("Dia da semana com maior receita total é")
display(dia_maior_receita)

# Mês com maior ticket médio no último ano
ultimo_ano = spark.sql("SELECT MAX(ano) as ultimo_ano FROM medalhao.gold.view_vendas_por_periodo").collect()[0]['ultimo_ano']

mes_maior_ticket = spark.sql(f"""
SELECT 
    mes,
    mes_nome,
    AVG(ticket_medio_brl) as ticket_medio
FROM medalhao.gold.view_vendas_por_periodo
WHERE ano = {ultimo_ano}
GROUP BY mes, mes_nome
ORDER BY ticket_medio DESC
LIMIT 1
""")

print(f"Mês com maior ticket médio no ano {ultimo_ano} é")
display(mes_maior_ticket)

- Usa tabelas as criadas nas células anteriores para mostrar quais são os pedidos entregues, agrupados por dt.ano, dt.trimestre, dt.mes, dt.mes_nome, dt.dia, dt.dia_da_semana_num
- Também calcula novas métricas como as presentes da coluna receita_total_brl(total das vendas em reais) até avaliacao_media(média das avaliações dos pedidos)
- Ordena o numero total de receita feita em cada dia da semana e pega o dia que fique no topo dessa ordenação em ordem decrescente, junto com a receita dele
- Ordena o ticket médio por mes do último ano registrado e pega o mês que fique no topo dessa ordenação em ordem decrescente, junto com seu ticket médio e o número correspondente ao mês

### 4.4 - Criação da "gold.view_top_produto":

In [0]:
# View para análise de performance de produtos
spark.sql("""
CREATE OR REPLACE VIEW medalhao.gold.view_top_produto AS
SELECT 
    fv.fk_produto as id_produto,
    MAX(fp.categoria_produto) as categoria_produto,
    COUNT(fv.id_item) as quantidade_vendida,
    COUNT(DISTINCT fv.id_pedido) as total_pedidos,
    CAST(ROUND(SUM(fv.valor_total_item_brl), 2) AS DECIMAL(12,2)) as receita_brl,
    CAST(ROUND(SUM(fv.valor_total_item_usd), 2) AS DECIMAL(12,2))as receita_usd,
    CAST(ROUND(AVG(fv.valor_produto_brl), 2) AS DECIMAL(12,2)) as preco_medio_brl,
    CAST(ROUND(AVG(fv.avaliacao_pedido), 2) AS DECIMAL(3,2)) as avaliacao_media,
    CAST(ROUND(AVG(fp.peso_produto_gramas), 2) AS DECIMAL(8,2)) as peso_medio_gramas
FROM medalhao.gold.ft_vendas_geral fv
LEFT JOIN medalhao.silver.ft_produtos fp ON fv.fk_produto = fp.id_produto
WHERE fv.status_pedido = 'entregue'
GROUP BY fv.fk_produto
""")

# Validação da view
top_produto_view = spark.sql("SELECT * FROM medalhao.gold.view_top_produto ORDER BY receita_brl DESC, avaliacao_media DESC LIMIT 10")
display(top_produto_view)

- Junção dos elementos entregues de ft_vendas_geral, com os dados contidos em ft_produtos, os quais "fv.fk_produto = fp.id_produto"(agrupados os que possuem mesma identificação), onde fv representa a tabela de vendas e fp a de produtos
- Também calcula novas métricas como as presentes da coluna receita_brl(receita total das vendas em reais) até peso_medio_gramas(peso médio do produto)
- Por fim, a view é ordenada pela receita e avaliação, da maior para menor, já que a área de Gestão de Produtos e Comercial deseja saber quais produtos e categorias geram mais receita e possuem melhor desempenho de vendas e avaliação

### 4.5 - Criação da "view_vendas_produtos_esteticos":

In [0]:
# View específica para categoria fashion usando CTE
spark.sql("""
CREATE OR REPLACE VIEW medalhao.gold.view_vendas_produtos_esteticos AS
WITH vendas_fashion AS (
    SELECT 
        fv.fk_tempo,
        fp.categoria_produto,
        fv.id_pedido,
        fv.id_item,
        fv.valor_total_item_brl,
        fv.valor_total_item_usd,
        fv.avaliacao_pedido
    FROM medalhao.gold.ft_vendas_geral fv
    LEFT JOIN medalhao.silver.ft_produtos fp ON fv.fk_produto = fp.id_produto
    WHERE fp.categoria_produto LIKE 'fashion%'
      AND fv.status_pedido = 'entregue'
),
vendas_agregadas AS (
    SELECT 
        vf.fk_tempo,
        vf.categoria_produto,
        COUNT(DISTINCT vf.id_pedido) as total_pedidos,
        COUNT(vf.id_item) as total_itens_vendidos,
        SUM(vf.valor_total_item_brl) as receita_total_brl,
        SUM(vf.valor_total_item_usd) as receita_total_usd,
        AVG(vf.avaliacao_pedido) as avaliacao_media
    FROM vendas_fashion vf
    GROUP BY vf.fk_tempo, vf.categoria_produto
)
SELECT 
    dt.ano,
    dt.mes,
    va.categoria_produto,
    va.total_pedidos,
    va.total_itens_vendidos,
    CAST(ROUND(va.receita_total_brl, 2) AS DECIMAL(12,2)) as receita_total_brl,
    CAST(ROUND(va.receita_total_usd, 2) AS DECIMAL(12,2)) as receita_total_usd,
    CAST(ROUND(va.receita_total_brl / NULLIF(va.total_itens_vendidos, 0), 2) AS DECIMAL(12,2)) as ticket_medio_brl,
    CAST(ROUND(va.receita_total_usd / NULLIF(va.total_itens_vendidos, 0), 2) AS DECIMAL(12,2)) as ticket_medio_usd,
    CAST(ROUND(va.avaliacao_media, 2) AS DECIMAL(3,2)) as avaliacao_media
FROM vendas_agregadas va
JOIN medalhao.gold.dm_tempo dt ON va.fk_tempo = dt.sk_tempo
ORDER BY dt.ano, dt.mes, va.categoria_produto
""")

# Validação da view
vendas_fashion_view = spark.sql("SELECT * FROM medalhao.gold.view_vendas_produtos_esteticos LIMIT 10")
display(vendas_fashion_view)

- Junção dos elementos entregues de ft_vendas_geral, com os dados contidos em ft_produtos(principalmente categoria_produto, que será filtrado mostrando apenas os que começarem com a palavra "fashion"), os quais "fv.fk_produto = fp.id_produto"(agrupados os que possuem mesma identificação), onde fv representa a tabela de vendas e fp a de produtos, isso para criar vendas_fashion
- Com vendas_fashion são feitas operações de contagem, soma e média entre as colunas de total_pedidos até avaliacao_media, criando vendas_agregadas
- Por fim, com vendas_agregadas forma-se a ordem das tabelas de acordo com a atividade, mantendo seus tipos de dados também, 